# Pipeline for calculating overbite

## Initialization

In [3]:
# Importing packages
import os
import torch
import re
import torchvision.transforms as T
from collections import defaultdict
from PIL import Image

In [1]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  1.13 ; cuda:  cu117
detectron2: 0.6


# Generatig png from ply file

In [2]:
%run Pipeline_code/Ply_To_Image.py

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
['Pipeline_data/Ply Files/Brunatest LowerJawScan.ply', 'Pipeline_data/Ply Files/Brunatest_UpperJawScan.ply']


/home/user/app/Pipeline_code/Ply_To_Image.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  vertices = torch.tensor(ply.vertices, dtype=torch.float32)


Brunatest LowerJawScan_0.png: Rotated 90° clockwise_1.png 1/2
Brunatest LowerJawScan_1.png: Flipped on X-axis and rotated 90° clockwise
Brunatest_UpperJawScan_0.png: Rotated 90° clockwise
Brunatest_UpperJawScan_1.png: Flipped on X-axis and rotated 90° clockwise


# Opdeling af billeder i Lower/Upper og Left/Right/Middle 

In [8]:
# %run Pipeline_code/Opdeling_og_flip_af_billeder.py

Loaded 2 images from Pipeline_data\Raw_data
✅ All images have dimensions: 3072x1024
✅ Processed 2 images and saved to Pipeline_data\Clean Data


# Indlæs den trænede model og kør modellen

In [30]:
%run Pipeline_code/Run_model.py

torch:  1.13 ; cuda:  cu117
detectron2: 0.6
[05/21 11:53:47 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from Pipeline_data/Model/model_0054999.pth ...
Model reloaded successfully!
My Test dataset was removed
✅ Final results saved to: Pipeline_data/Predicted_keypoints.csv


# Pixelmatrix på data 

In [31]:
%run Pipeline_code/Pixelmatrix.py

100%|███████████████████████████████████████████████████████████████████████| 304/304 [00:26<00:00, 11.63it/s]

Saved refined results to Pipeline_data/Output_after_pixel_matrix/Predicted_keypoints_pixel_matrix.csv


# Overbite

In [34]:
%run Pipeline_code/Overbite.py

Loading data...
Extracting metadata from filenames...
Separating upper and lower jaw entries...
Merging upper and lower jaw data by Base_ID and Suffix...
Found 152 matched upper-lower pairs.
Calculating overbite for each pair...
      Base_ID Suffix  Y_upper  Y_lower  overbite_pixel  overbite_mm
0    013FHA7K   left      276      369              93         7.44
1    013FHA7K  right      276      369              93         7.44
2    013NUWYR   left      271      243             -28        -2.24
3    013NUWYR  right      272      235             -37        -2.96
4    013NXP1H   left      268      357              89         7.12
..        ...    ...      ...      ...             ...          ...
147  LA67YYVJ  right      352      436              84         6.72
148  LGADV15G   left      261      350              89         7.12
149  LGADV15G  right      262      353              91         7.28
150  MNIAB8K3   left      266      336              70         5.60
151  MNIAB8K3  right   